In [ ]:
pip install pandas openpyxl

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import gc

# Đọc dữ liệu từ Google Sheet dưới dạng xlsx
file_path = 'https://drive.google.com/uc?id=1Ahsx819pr1_uoBzYpAQ3y7jCvrzrrEpj' # "1izHHnc_CzggaFDNgc2XhAMiA_K7D-_D4" là 1 triệu dòng "1Ahsx819pr1_uoBzYpAQ3y7jCvrzrrEpj" là 10 nghìn dòng
df = pd.read_excel(file_path, engine='openpyxl')

# Hàm tiền xử lý văn bản
def clean_text(text):
    text = re.sub(r'\W', ' ', text)  # Loại bỏ dấu câu
    text = text.lower()  # Chuyển thành chữ thường
    text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng thừa
    return text

# Đảm bảo cột 'text' và 'label' không có giá trị NaN
df = df.dropna(subset=['text', 'label'])

# Thay thế NaN trong 'text' bằng chuỗi rỗng và xử lý văn bản
df['text'] = df['text'].fillna('').apply(lambda x: clean_text(x) if isinstance(x, str) else '')

# Tách dữ liệu thành tập huấn luyện và kiểm thử
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Biến đổi văn bản thành vector số
vectorizer = TfidfVectorizer(max_features=1000)                                # Giảm số còn 500 nếu sử dụng link 10tr, 1000 cho link 10k
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Xây dựng mô hình Deep Learning với kích thước nhỏ hơn
dl_model = Sequential()
dl_model.add(Dense(64, activation='relu', input_shape=(X_train_vectorized.shape[1],)))
dl_model.add(Dropout(0.5))
dl_model.add(Dense(32, activation='relu'))
dl_model.add(Dense(1, activation='sigmoid'))

# Biên dịch mô hình
dl_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình Deep Learning với batch size nhỏ hơn và không dùng toarray()
dl_model.fit(X_train_vectorized, y_train, epochs=5, batch_size=16, validation_data=(X_test_vectorized, y_test))

# Đánh giá mô hình Deep Learning
dl_loss, dl_accuracy = dl_model.evaluate(X_test_vectorized, y_test)
print(f'Accuracy Deep Learning: {dl_accuracy:.4f}')

# Dọn dẹp bộ nhớ sau khi hoàn thành
gc.collect()

# Dự đoán nhãn cho một câu mới bằng Deep Learning
def predict_label_dl(text):
    clean_text_input = clean_text(text)
    text_vectorized = vectorizer.transform([clean_text_input])
    prediction = dl_model.predict(text_vectorized)
    return 1 if prediction >= 0.5 else 0

new_text = "Đây là câu ví dụ để kiểm tra."
predicted_label_dl = predict_label_dl(new_text)
print(f'Nhãn dự đoán với Deep Learning: {predicted_label_dl}')

# Lưu kết quả dự đoán vào tệp Excel
results = pd.DataFrame({
    'text': X_test,
    'actual_label': y_test,
    'predicted_label_dl': (dl_model.predict(X_test_vectorized) >= 0.5).astype(int).flatten()
})
results.to_excel('ket_qua_du_doan.xlsx', index=False)
print("Kết quả dự đoán đã được lưu vào 'ket_qua_du_doan.xlsx'")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6850 - loss: 0.5805 - val_accuracy: 0.8919 - val_loss: 0.2769
Epoch 2/5
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9003 - loss: 0.2469 - val_accuracy: 0.8972 - val_loss: 0.2693
Epoch 3/5
528/528 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9263 - loss: 0.1978 - val_accuracy: 0.8934 - val_loss: 0.2759
Epoch 4/5
528/528 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9375 - loss: 0.1783 - val_accuracy: 0.8948 - val_loss: 0.2842
Epoch 5/5
528/528 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9496 - loss: 0.1411 - val_accuracy: 0.8948 - val_loss: 0.2979
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9045 - loss: 0.2708
Accuracy Deep Learning: 0.8948
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Nhãn dự đoán với Deep Learning: 0
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Kết quả dự đoán đã được lưu vào 'ket_qua_du_doan.xlsx'


In [5]:
def predict_label_dl(text):
    clean_text_input = clean_text(text)
    text_vectorized = vectorizer.transform([clean_text_input])
    prediction = dl_model.predict(text_vectorized.toarray())
    return 1 if prediction >= 0.5 else 0

# Nhập câu văn bản từ người dùng
new_text = input("Nhập câu văn bản để kiểm tra: ")

# Dự đoán nhãn cho câu văn bản mới
predicted_label_dl = predict_label_dl(new_text)

# Hiển thị kết quả dự đoán
if predicted_label_dl == 1:
    print("Câu văn bản này có dấu hiệu toxic.")
else:

    print("Câu văn bản này không có dấu hiệu toxic.")

Nhập câu văn bản để kiểm tra: đi chết đi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Câu văn bản này có dấu hiệu toxic.
